In [1]:
from utils import load_dataframe
import fastai

In [2]:
dataset_description_path = "dataset_description.yaml"

In [3]:
#Generate dataset from description
tb_df = load_dataframe(dataset_description_path)

In [4]:
tb_df

,file,path,abnormal,tuberculosis
0,00000562_000.png,dataset/anormal-xr-no_tb/00000562_000.png,1,0
1,00000199_002.png,dataset/anormal-xr-no_tb/00000199_002.png,1,0
2,00000628_000.png,dataset/anormal-xr-no_tb/00000628_000.png,1,0
3,00000897_000.png,dataset/anormal-xr-no_tb/00000897_000.png,1,0
4,00000830_006.png,dataset/anormal-xr-no_tb/00000830_006.png,1,0
...,...,...,...,...
15753,Tuberculosis-49.png,dataset/TB_Chest_Radiography_Database/Tubercul...,1,1
15754,Tuberculosis-127.png,dataset/TB_Chest_Radiography_Database/Tubercul...,1,1
15755,Tuberculosis-166.png,dataset/TB_Chest_Radiography_Database/Tubercul...,1,1
15756,Tuberculosis-237.png,dataset/TB_Chest_Radiography_Database/Tubercul...,1,1
